In [ ]:
!pip install sentence-transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=0abe0243d6c098966e73f307637c500ebdfb692ac7562f970115406d4e195c93
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
train_data = pd.read_csv("/content/train - train.csv")
test_data = pd.read_csv("/content/evaluation - evaluation.csv")

In [ ]:
train_shape = np.shape(train_data)
train_len = train_shape[0]
train_len

2061

In [ ]:
test_shape = np.shape(test_data)
test_len = test_shape[0]
test_len

9000

In [ ]:
train_data.head()

,text,reason,label
0,this is an amazing app for online classes!but,good app for conducting online classes,1
1,very practical and easy to use,app is user-friendly,1
2,this app is very good for video conferencing.,good for video conferencing,1
3,i can not download this zoom app,unable to download zoom app,1
4,i am not able to download this app,want to download the app,1


In [ ]:
test_data.head()

,text,reason,label
0,the app is crashing when i play a vedio,app crashes during playback,1
1,but i want to connect it to the tv from one de...,want compatibility with more smart televisions,0
2,very helpful when and home working remotley,good app for work,0
3,this zoom so called and missed call and mobile...,receiving incorrect phone number message,0
4,one of my favorite apps,good for spending time,0


In [ ]:
train_data.text.values

array(['this is an amazing app for online classes!but',
       'very practical and easy to use',
       'this app is very good for video conferencing.', ...,
       'hello,\ndisney+ must also be installed on chromecast',
       'it is a shame that disney+ does not work on tv box devices.',
       'i have to close and reload the app over and over again until the sound works.'],
      dtype=object)

In [ ]:
text_embeddings = model.encode(train_data.text.values)
text_embeddings.shape

(2061, 768)

In [ ]:
text_embeddings_test = model.encode(test_data.text.values)
text_embeddings_test.shape

(9000, 768)

In [ ]:
reason_embeddings = model.encode(train_data.reason.values)
reason_embeddings.shape

(2061, 768)

In [ ]:
reason_embeddings_test = model.encode(test_data.reason.values)
reason_embeddings_test.shape

(9000, 768)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

training_pred=[]
for i in range(0,train_len-1):

  a=(cosine_similarity(
      [text_embeddings[i]],
      reason_embeddings[i:i+1]
  ))
  # print(a)
  threshold = 0.5
  t=a
  # print(t)
  if t< 0.5 :
    training_pred.append(0)
  if t>= 0.5 :
    training_pred.append(1)

In [ ]:
test_pred=[]
for i in range(0,test_len-1):

  a=(cosine_similarity(
      [text_embeddings_test[i]],
      reason_embeddings_test[i:i+1]
  ))
  # print(a)
  threshold = 0.5
  t=a
  # print(t)
  if t< 0.5 :
    test_pred.append(0)
  if t>= 0.5 :
    test_pred.append(1)

In [ ]:
training_pred=cos_sim(text_embeddings,reason_embeddings)

In [ ]:
np.shape(a)

(1, 1)

In [ ]:
training_pred

[1]

In [ ]:
def calculate_accuracy(predictions, ground_truth):
    """
    Calculates the accuracy given a list of predicted values and a list of ground truth values.
    """
    num_correct = 0
    for pred, gt in zip(predictions, ground_truth):
        if pred == gt:
            num_correct += 1

    accuracy = num_correct / len(predictions)
    return accuracy

In [ ]:
# Accuacy for dataset without negatives
accuracy=calculate_accuracy(training_pred, train_data["label"])
accuracy

0.7480582524271845

In [ ]:
# Accuacy for eval dataset 
accuracy=calculate_accuracy(test_pred, test_data["label"])
accuracy

0.6095121680186687